# Load Data and analyse features

In [119]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
#import the json split up csv
train = pd.read_csv("../train-v2.csv")
train.head()

,channelGrouping,fullVisitorId,visitId,visitNumber,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.hits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,year,month,hour,dayofweek,isWeekend
0,Organic Search,3162355547410993243,1508198450,1,Firefox,desktop,False,Windows,NaN,Europe,Germany,NaN,NaN,NaN,Western Europe,1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,water bottle,organic,NaN,google,2017,10,0,1,0
1,Referral,8934116514970143966,1508176307,6,Chrome,desktop,False,Chrome OS,Cupertino,Americas,United States,San Francisco-Oakland-San Jose CA,NaN,California,Northern America,2,2.0,2.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,2017,10,17,0,0
2,Direct,7992466427990357681,1508201613,1,Chrome,mobile,True,Android,NaN,Americas,United States,NaN,windjammercable.net,NaN,Northern America,2,2.0,1.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(none),NaN,(direct),2017,10,0,1,0
3,Organic Search,9075655783635761930,1508169851,1,Chrome,desktop,False,Windows,NaN,Asia,Turkey,NaN,unknown.unknown,NaN,Western Asia,2,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google,2017,10,16,0,0
4,Organic Search,6960673291025684308,1508190552,1,Chrome,desktop,False,Windows,NaN,Americas,Mexico,NaN,prod-infinitum.com.mx,NaN,Central America,2,2.0,1.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google,2017,10,21,0,0


In [120]:
print(str(train.shape[0]- 903653) + ": new entries")
print(43-train.shape[1])


804684: new entries
6


In [121]:
#Goes through each feature and removes if has less than 2 unique values or more than 50% of its entries are null
def uniqValues(df):
    for colName in list(df):
        nuniq = df[colName].nunique()
        percNull = float(df[colName].isnull().sum().sum())/df.shape[0]
        print('{:<45}  {:>15} {:>15}'.format(colName, nuniq, percNull))
        if nuniq <2 or percNull > .5:
            df = df.drop(columns = [colName])
            print("deleted: " + colName)
        
    return df


train = train.drop(columns=['geoNetwork.networkDomain'])
train = uniqValues(train)



channelGrouping                                              8             0.0
fullVisitorId                                          1371486             0.0
visitId                                                1665802             0.0
visitNumber                                                457             0.0
device.browser                                             128 7.02437516719e-06
device.deviceCategory                                        3             0.0
device.isMobile                                              2             0.0
device.operatingSystem                                      23 0.0069160827167
geoNetwork.city                                            954  0.584677379229
deleted: geoNetwork.city
geoNetwork.continent                                         5 0.00147336269132
geoNetwork.country                                         227 0.00147336269132
geoNetwork.metro                                           121  0.772596390525
deleted: geoNetwork.met

In [122]:
list(train)

['channelGrouping',
 'fullVisitorId',
 'visitId',
 'visitNumber',
 'device.browser',
 'device.deviceCategory',
 'device.isMobile',
 'device.operatingSystem',
 'geoNetwork.continent',
 'geoNetwork.country',
 'geoNetwork.subContinent',
 'totals.hits',
 'totals.pageviews',
 'totals.sessionQualityDim',
 'trafficSource.medium',
 'trafficSource.source',
 'year',
 'month',
 'hour',
 'dayofweek',
 'isWeekend']

# Merge in Country Dataset

In [123]:
#Country dataset: https://www.kaggle.com/fernandol/countries-of-the-world



In [124]:
#population
#literacy
#Phones (per 1,000)
#climate
#GDP
countries = pd.read_csv("../countries.csv")
#Strip blank space and The in country names to match format of Kaggle
countries['Country'] = countries['Country'].str.strip()
countries['Country'] = countries['Country'].str.replace(", The", "", regex =False)

#Convert values to floats
countries['Literacy (%)'] = countries['Literacy (%)'].str.replace(",", ".", regex =False)
countries['Literacy (%)'] = countries['Literacy (%)'].astype(float)
countries['Phones (per 1000)'] = countries['Phones (per 1000)'].str.replace(",", ".", regex =False)
countries['Phones (per 1000)'] = countries['Phones (per 1000)'].astype(float)
countries['Pop. Density (per sq. mi.)'] = countries['Pop. Density (per sq. mi.)'].str.replace(",", ".", regex =False)
countries['Pop. Density (per sq. mi.)'] = countries['Pop. Density (per sq. mi.)'].astype(float)

countries = countries.drop(columns=['Region','Area (sq. mi.)','Coastline (coast/area ratio)',
'Net migration','Infant mortality (per 1000 births)','Arable (%)','Crops (%)','Other (%)','Birthrate',
                                    'Deathrate','Agriculture','Industry','Service', 'Climate'])


replLocList = [32,109,110,53,123,208,172,173,175,220,147,174,28,58,45,46,39,153,49,178,167,171,136]
CountryChangedList = ['Myanmar (Burma)', 'North Korea', 'South Korea','Czechia','Macedonia (FYROM)','Turks & Caicos Islands',
    'St. Kitts & Nevis','St. Lucia','St. Vincent & Grenadines','U.S. Virgin Islands','Caribbean Netherlands','St. Pierre & Miquelon',
    'British Virgin Islands','Timor-Leste','Congo - Kinshasa','Congo - Brazzaville','Central African Republic','Northern Mariana Islands', 
    'C\xc3\xb4te d\xe2\x80\x99Ivoire','S\xc3\xa3o Tom\xc3\xa9 & Pr\xc3\xadncipe','R\xc3\xa9union','St. Helena','Micronesia']
for cnt, replLoc in enumerate(replLocList): 
    print(str(replLoc) + ": Changing - " +  countries['Country'].iloc[replLoc] + " for " + CountryChangedList[cnt])
    
    
countries['Country'].iloc[32] = 'Myanmar (Burma)'
countries['Country'].iloc[109] = 'North Korea'
countries['Country'].iloc[110] = 'South Korea'
countries['Country'].iloc[53] = 'Czechia'
countries['Country'].iloc[123] = 'Macedonia (FYROM)'
countries['Country'].iloc[208] = 'Turks & Caicos Islands'
countries['Country'].iloc[172] = 'St. Kitts & Nevis'
countries['Country'].iloc[173] = 'St. Lucia'
countries['Country'].iloc[175] = 'St. Vincent & Grenadines'
countries['Country'].iloc[220] = 'U.S. Virgin Islands'
countries['Country'].iloc[147] = 'Caribbean Netherlands'
countries['Country'].iloc[174] = 'St. Pierre & Miquelon'
countries['Country'].iloc[28] = 'British Virgin Islands'
countries['Country'].iloc[58] = 'Timor-Leste'
countries['Country'].iloc[45] = 'Congo - Kinshasa'
countries['Country'].iloc[46] = 'Congo - Brazzaville'
countries['Country'].iloc[39] = 'Central African Republic'
countries['Country'].iloc[153] = 'Northern Mariana Islands'
countries['Country'].iloc[49] = 'C\xc3\xb4te d\xe2\x80\x99Ivoire'
countries['Country'].iloc[178] = 'S\xc3\xa3o Tom\xc3\xa9 & Pr\xc3\xadncipe'
countries['Country'].iloc[167] = 'R\xc3\xa9union'
countries['Country'].iloc[171] = 'St. Helena'
countries['Country'].iloc[136] = 'Micronesia'

comb = pd.merge(train, countries, left_on = "geoNetwork.country", right_on = "Country", how = 'left')
comb = comb.drop(columns=['Country'])

32: Changing - Burma for Myanmar (Burma)
109: Changing - Korea, North for North Korea
110: Changing - Korea, South for South Korea
53: Changing - Czech Republic for Czechia
123: Changing - Macedonia for Macedonia (FYROM)
208: Changing - Turks & Caicos Is for Turks & Caicos Islands
172: Changing - Saint Kitts & Nevis for St. Kitts & Nevis
173: Changing - Saint Lucia for St. Lucia
175: Changing - Saint Vincent and the Grenadines for St. Vincent & Grenadines
220: Changing - Virgin Islands for U.S. Virgin Islands
147: Changing - Netherlands Antilles for Caribbean Netherlands
174: Changing - St Pierre & Miquelon for St. Pierre & Miquelon
28: Changing - British Virgin Is. for British Virgin Islands
58: Changing - East Timor for Timor-Leste
45: Changing - Congo, Dem. Rep. for Congo - Kinshasa
46: Changing - Congo, Repub. of the for Congo - Brazzaville
39: Changing - Central African Rep. for Central African Republic
153: Changing - N. Mariana Islands for Northern Mariana Islands
49: Changing -

# Normalize numerical data

In [125]:

from sklearn import preprocessing
from scipy.stats import zscore
#  = a                    # initialise array for zscores
# z[~np.isnan(a)] = zscore(a[~np.isnan(a)])
# pandas.DataFrame({'a':a,'Zscore':z})

#go through list of numberical columns to normalize
for countryCol in list(countries):
    if countryCol != 'Country':
        x = comb[countryCol]
        temp = comb[countryCol]
        x[~np.isnan(temp)] = zscore(temp[~np.isnan(temp)])
        comb[countryCol] = x



/Users/cassidylitch/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# One-Hot-Encode

In [126]:
comb['geoNetwork.subContinent'].unique()

array(['Western Europe', 'Northern America', 'Western Asia',
       'Central America', 'Northern Europe', 'Southern Asia',
       'Southeast Asia', 'Eastern Europe', 'South America',
       'Eastern Asia', 'Southern Europe', 'Australasia', 'Central Asia',
       nan, 'Northern Africa', 'Eastern Africa', 'Southern Africa',
       'Western Africa', 'Caribbean', 'Middle Africa', 'Melanesia',
       'Micronesian Region', 'Polynesia'], dtype=object)

In [127]:
train = pd.get_dummies(comb,columns=['geoNetwork.continent', 'geoNetwork.subContinent'])


In [117]:
comb.head()

,channelGrouping,fullVisitorId,visitId,visitNumber,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.country,totals.hits,totals.pageviews,totals.sessionQualityDim,trafficSource.medium,trafficSource.source,year,month,hour,dayofweek,isWeekend,Population,Pop. Density (per sq. mi.),GDP ($ per capita),Literacy (%),Phones (per 1000),geoNetwork.continent_Africa,geoNetwork.continent_Americas,geoNetwork.continent_Asia,geoNetwork.continent_Europe,geoNetwork.continent_Oceania,geoNetwork.subContinent_Australasia,geoNetwork.subContinent_Caribbean,geoNetwork.subContinent_Central America,geoNetwork.subContinent_Central Asia,geoNetwork.subContinent_Eastern Africa,geoNetwork.subContinent_Eastern Asia,geoNetwork.subContinent_Eastern Europe,geoNetwork.subContinent_Melanesia,geoNetwork.subContinent_Micronesian Region,geoNetwork.subContinent_Middle Africa,geoNetwork.subContinent_Northern Africa,geoNetwork.subContinent_Northern America,geoNetwork.subContinent_Northern Europe,geoNetwork.subContinent_Polynesia,geoNetwork.subContinent_South America,geoNetwork.subContinent_Southeast Asia,geoNetwork.subContinent_Southern Africa,geoNetwork.subContinent_Southern Asia,geoNetwork.subContinent_Southern Europe,geoNetwork.subContinent_Western Africa,geoNetwork.subContinent_Western Asia,geoNetwork.subContinent_Western Europe
0,Organic Search,3162355547410993243,1508198450,1,Firefox,desktop,False,Windows,Germany,1,1.0,1.0,organic,google,2017,10,0,1,0,-0.559015,0.042366,0.178387,0.545663,0.300013,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Referral,8934116514970143966,1508176307,6,Chrome,desktop,False,Chrome OS,United States,2,2.0,2.0,referral,sites.google.com,2017,10,17,0,0,0.260052,-0.225730,0.925696,0.370510,1.016827,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Direct,7992466427990357681,1508201613,1,Chrome,mobile,True,Android,United States,2,2.0,1.0,(none),(direct),2017,10,0,1,0,0.260052,-0.225730,0.925696,0.370510,1.016827,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Organic Search,9075655783635761930,1508169851,1,Chrome,desktop,False,Windows,Turkey,2,2.0,1.0,organic,google,2017,10,16,0,0,-0.604546,-0.146334,-1.352865,-0.549045,-0.941094,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Organic Search,6960673291025684308,1508190552,1,Chrome,desktop,False,Windows,Mexico,2,2.0,1.0,organic,google,2017,10,21,0,0,-0.464122,-0.194213,-1.184354,-0.049858,-1.214923,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Is Chrome Feature

In [128]:
train.loc[train["device.browser"]!="Chrome","device.browser"] = 0
train.loc[train["device.browser"]=="Chrome", "device.browser"] = 1
train.rename(columns={'device.browser': 'device.isChrome'}, inplace=True)


In [ ]:
pd.to